In [67]:
import numpy as np
import pandas as pd
import scipy.spatial.distance as dist
from sympy import *
from collections import defaultdict

In [68]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [73]:
class PageRank:
    def __init__(self, edge_list, init_val=None, rounding_digit = 3, n_iter = 10):
        self.edge_list = edge_list
        self.prepare_page_dict()
        self.rounding_digit = rounding_digit
        self.n_iter = n_iter
        if init_val:
            self.init_val = init_val
        else:
            self.init_val = [1/len(self.page_list) for x in self.page_list]
    
    def prepare_page_dict(self):
        self.page_source_dict = defaultdict(list)
        self.page_dest_dict = defaultdict(list)
        self.page_list = []
        for s,d in self.edge_list:
            self.page_source_dict[s].append(d)
            self.page_dest_dict[d].append(s)
            
            if s not in self.page_list:
                self.page_list.append(s)
            if d not in self.page_list:
                self.page_list.append(d)
        self.page_list.sort()
        
    def evaluate(self):
        print('Rank of different webpages:\n')
#         print(self.page_source_dict)
#         print(self.page_dest_dict)
#         print(self.page_list)
        for page in self.page_list:
            if self.page_dest_dict[page]:
                s = ''
                for dest in self.page_dest_dict[page]:
                    s += f'R({dest})/{len(self.page_source_dict[dest])} +'
                s = s[:-1]
                display(Symbol(f'R({page})={s}'))
        self.iteration_matrix = pd.DataFrame(index=self.page_list)
        
        vals = {x:y for x, y in zip(self.page_list, self.init_val)}
        print(f'\nInitial values:\n{vals}')
        
        self.iteration_matrix['Initial Values'] = self.init_val
        n_iteration = 1
        cur_val = vals
        rank_list = []
        prev_rank_list = []
        while n_iteration <= self.n_iter:
            vals = {}
            print(f'\nIteration {n_iteration}:\n')
            for page in self.page_list: 
                if self.page_dest_dict[page]:
                    s = ''
                    rank = 0
                    for dest in self.page_dest_dict[page]:
                        #print(cur_val)
                        s += f'{cur_val[dest]}/{len(self.page_source_dict[dest])} +'
                        rank+= cur_val[dest]/len(self.page_source_dict[dest])
                    s = s[:-1]
                    
                    display(Symbol(f'R({page})={s} = {round(rank, self.rounding_digit)}'))
                else:
                    rank = cur_val[page]
                    display(Symbol(f'R({page}) = {rank}'))
                rank = round(rank, self.rounding_digit)
                vals[page] = rank
                rank_list.append(rank)
            #print(rank_list)
            
            self.iteration_matrix[f'Iteration {n_iteration}'] = rank_list
            if prev_rank_list == rank_list:
                print('\n Converged')
                break
            n_iteration +=1
            
            prev_rank_list = rank_list
            rank_list = []
            cur_val = vals
            print('--------------------------------------------------------------')
        self.iteration_matrix['Page rank'] = self.iteration_matrix[self.iteration_matrix.columns[-1]]. \
                                                    rank(method='dense', ascending=False).astype(int)
        display(self.iteration_matrix)

In [74]:
edge_list = [('A', 'C'),('C', 'A'),('A','B'), ('C', 'B'), ('B','D'), ('C','D'), ('D','C')]
PageRank(edge_list).evaluate()

Rank of different webpages:



R(A)=R(C)/3 

R(B)=R(A)/2 +R(C)/3 

R(C)=R(A)/2 +R(D)/1 

R(D)=R(B)/1 +R(C)/3 

...

R(B)=0.124/2 +0.376/3  = 0.187

R(C)=0.124/2 +0.311/1  = 0.373

R(D)=0.187/1 +0.376/3  = 0.312

--------------------------------------------------------------


,Initial Values,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Page rank
A,0.25,0.083,0.125,0.125,0.132,0.118,0.128,0.124,0.125,0.124,0.125,4
B,0.25,0.208,0.167,0.187,0.195,0.184,0.187,0.188,0.187,0.187,0.187,3
C,0.25,0.375,0.374,0.396,0.354,0.385,0.372,0.376,0.373,0.376,0.373,1
D,0.25,0.333,0.333,0.292,0.319,0.313,0.312,0.311,0.313,0.311,0.312,2
